In [ ]:
pip install torch transformers datasets scikit-learn pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load dataset
df = pd.read_csv("final_synthetic_text_classification_dataset.csv")

# Encode text labels into numbers
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["Category"])  # New 'label' column

# Convert to Hugging Face dataset format
dataset = Dataset.from_pandas(df)

# Split into train/test sets
dataset = dataset.train_test_split(test_size=0.2)

# Save label mapping for later inference
label_mapping = {i: label for i, label in enumerate(label_encoder.classes_)}
print("Label Mapping:", label_mapping)


Label Mapping: {0: 'Daily Routine', 1: 'Daily Routine & Diet', 2: 'Daily Routine & Diet & Others', 3: 'Daily Routine & Diet & Stress Mental Well-being', 4: 'Daily Routine & Diet & Stress Mental Well-being & Others', 5: 'Daily Routine & Others', 6: 'Daily Routine & Stress Mental Well-being', 7: 'Daily Routine & Stress Mental Well-being & Others', 8: 'Diet', 9: 'Diet & Others', 10: 'Diet & Stress Mental Well-being', 11: 'Diet & Stress Mental Well-being & Others', 12: 'Exercise', 13: 'Exercise & Daily Routine', 14: 'Exercise & Daily Routine & Diet', 15: 'Exercise & Daily Routine & Diet & Others', 16: 'Exercise & Daily Routine & Diet & Stress Mental Well-being', 17: 'Exercise & Daily Routine & Diet & Stress Mental Well-being & Others', 18: 'Exercise & Daily Routine & Others', 19: 'Exercise & Daily Routine & Stress Mental Well-being', 20: 'Exercise & Daily Routine & Stress Mental Well-being & Others', 21: 'Exercise & Diet', 22: 'Exercise & Diet & Others', 23: 'Exercise & Diet & Stress Menta

In [ ]:
from transformers import DistilBertTokenizer

MODEL_NAME = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
def tokenize_function(examples):
    tokens = tokenizer(examples["Sentence"], truncation=True, padding="max_length")
    tokens["labels"] = examples["label"]  # Add numeric labels
    return tokens

dataset = dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [ ]:
from transformers import DistilBertForSequenceClassification

# Get unique labels
labels = list(df["Category"].unique())
num_labels = len(label_encoder.classes_)  # Ensure model knows number of classes
model = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)



model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
!pip install --upgrade --force-reinstall wandb



  Using cached wandb-0.19.8-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached platformdirs-4.3.6-py3-none-any.whl.metadata (11 kB)
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached psutil-7.0.0-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (22 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached sentry_sdk-2.22.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached setproctitle-1.3.5-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.many

In [ ]:



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer
)

trainer.train()



<ipython-input-7-cc9ae2cba04d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mittalyas1234 (mittalyas1234-jaypee-institute-of-information-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.630671
2,1.267200,0.168912
3,0.212700,0.099263


TrainOutput(global_step=1050, training_loss=0.7108490426199777, metrics={'train_runtime': 489.5961, 'train_samples_per_second': 17.157, 'train_steps_per_second': 2.145, 'total_flos': 1113301621555200.0, 'train_loss': 0.7108490426199777, 'epoch': 3.0})

In [ ]:
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")


('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/vocab.txt',
 'fine_tuned_model/added_tokens.json')

In [ ]:
# prompt: how we know accuracy of model

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

# Access specific metrics
accuracy = eval_results.get("eval_accuracy")
if accuracy:
    print(f"Accuracy: {accuracy}")
else:
    print("Accuracy not found in evaluation results.")


#Example to get other metrics like precision, recall, F1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

predictions = trainer.predict(dataset["test"])
predicted_labels = predictions.predictions.argmax(-1)
true_labels = dataset["test"]["label"]


accuracy = accuracy_score(true_labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")



{'eval_loss': 0.09926287829875946, 'eval_runtime': 10.7053, 'eval_samples_per_second': 65.388, 'eval_steps_per_second': 8.22, 'epoch': 3.0}
Accuracy not found in evaluation results.
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0


In [ ]:
# prompt: Accuracy: 1.0
# Precision: 1.0
# Recall: 1.0
# F1-score: 1.0
# it is giving this ?

# ... (Your existing code)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

# Access specific metrics - improved handling for missing keys
accuracy = eval_results.get("eval_accuracy", None)
if accuracy is not None:
    print(f"Accuracy: {accuracy}")
else:
    print("Accuracy not found in evaluation results.")

#Example to get other metrics like precision, recall, F1-score - improved handling of potential errors

try:
    predictions = trainer.predict(dataset["test"])
    predicted_labels = predictions.predictions.argmax(-1)
    true_labels = dataset["test"]["label"]


    accuracy = accuracy_score(true_labels, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted', zero_division=0) #handle zero division

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")

except Exception as e:
    print(f"An error occurred during metric calculation: {e}")


{'eval_loss': 0.09926287829875946, 'eval_runtime': 10.2526, 'eval_samples_per_second': 68.275, 'eval_steps_per_second': 8.583, 'epoch': 3.0}
Accuracy not found in evaluation results.
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0
